<a href="https://colab.research.google.com/github/DADADAVE80/qualcomm-ai-hub/blob/main/MobileNetV3_model_QAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install "qai-hub[torch]"

In [2]:
import numpy as np
import requests
import torch
from PIL import Image
from torchvision.models import mobilenet_v3_small
import qai_hub as hub

from google.colab import userdata
qai_key = userdata.get('QAI_API_TOKEN')
!qai-hub configure --api_token {qai_key}

# Using pre-trained MobileNet
torch_model = mobilenet_v3_small(pretrained=True)
torch_model.eval()

# Trace model
input_shape = (1, 3, 224, 224)
example_input = torch.rand(input_shape)
traced_torch_model = torch.jit.trace(torch_model, example_input)

# Compile model
compile_job = hub.submit_compile_job(
    model=traced_torch_model,
    device=hub.Device("Samsung Galaxy S24 (Family)"),
    input_specs=dict(image=input_shape),
    options="--target_runtime tflite",
)
assert isinstance(compile_job, hub.CompileJob)

# Profile on cloud-hosted device
target_model = compile_job.get_target_model()
assert isinstance(target_model, hub.Model)
profile_job = hub.submit_profile_job(
    model=target_model,
    device=hub.Device("Samsung Galaxy S24 (Family)"),
)
assert isinstance(profile_job, hub.ProfileJob)

qai-hub configuration saved to /root/.qai_hub/client.ini
==================== /root/.qai_hub/client.ini ====================
[api]
api_token = 32873ff887c548be7fb31779f6088e0ede7af401
api_url = https://app.aihub.qualcomm.com
web_url = https://app.aihub.qualcomm.com
verbose = True




/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 46.6MB/s]


Uploading tmp3wfctxdo.pt


100%|██████████| 10.2M/10.2M [00:00<00:00, 16.8MB/s]


Scheduled compile job (j56214qyg) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/j56214qyg/

Waiting for compile job (j56214qyg) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          
Scheduled profile job (jpvqwkzrg) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jpvqwkzrg/



In [3]:
# Run inference on cloud-hosted device
sample_image_url = (
    "https://qaihub-public-assets.s3.us-west-2.amazonaws.com/apidoc/input_image1.jpg"
)
response = requests.get(sample_image_url, stream=True)
response.raw.decode_content = True
image = Image.open(response.raw).resize((224, 224))
input_array = np.expand_dims(
    np.transpose(np.array(image, dtype=np.float32) / 255.0, (2, 0, 1)), axis=0
)

# Run inference using the on-device model on the input image
inference_job = hub.submit_inference_job(
    model=target_model,
    device=hub.Device("Samsung Galaxy S24 (Family)"),
    inputs=dict(image=[input_array]),
)
on_device_output = inference_job.download_output_data()

# Post-processing the on-device output
output_name = list(on_device_output.keys())[0]
out = on_device_output[output_name][0]
on_device_probabilities = np.exp(out) / np.sum(np.exp(out), axis=1)

# Read the class labels for imagenet
sample_classes = "https://qaihub-public-assets.s3.us-west-2.amazonaws.com/apidoc/imagenet_classes.txt"
response = requests.get(sample_classes, stream=True)
response.raw.decode_content = True
categories = [str(s.strip()) for s in response.raw]

# Print top five predictions for the on-device model
print("Top-5 On-Device predictions:")
top5_classes = np.argsort(on_device_probabilities[0], axis=0)[-5:]
for c in reversed(top5_classes):
    print(f"{c} {categories[c]:20s} {on_device_probabilities[0][c]:>6.1%}")

# Download model
target_model = compile_job.get_target_model()
target_model.download("mobilenet_v3.tflite")

Uploading dataset: 154kB [00:00, 365kB/s]                            


Scheduled inference job (j5ww96mm5) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/j5ww96mm5/

Waiting for inference job (j5ww96mm5) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          


tmpg61ssoqg.h5: 100%|██████████| 14.4k/14.4k [00:00<00:00, 6.79MB/s]


Top-5 On-Device predictions:
967 b'espresso'           87.6%
968 b'cup'                 7.9%
504 b'coffee mug'          3.5%
969 b'eggnog'              0.3%
849 b'teapot'              0.2%


mobilenet_v3.tflite: 100%|██████████| 9.71M/9.71M [00:00<00:00, 39.1MB/s]

Downloaded model to mobilenet_v3.tflite


'mobilenet_v3.tflite'